In [53]:
import requests 
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import csv
from io import StringIO

In [54]:
ishare_df=pd.read_csv('ishare.csv')
ishare_df['link']

0      https://www.ishares.com/us/products/239726/ish...
1      https://www.ishares.com/us/products/244049/ish...
2      https://www.ishares.com/us/products/239458/ish...
3      https://www.ishares.com/us/products/239706/ish...
4      https://www.ishares.com/us/products/239763/ish...
                             ...                        
430    https://www.ishares.com/us/products/328091/ish...
431    https://www.ishares.com/us/products/337460/ish...
432    https://www.ishares.com/us/products/332281/ish...
433    https://www.ishares.com/us/products/328179/ish...
434                                                  NaN
Name: link, Length: 435, dtype: object

In [55]:
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
#     'Accept-Language': 'en-US,en;q=0.9',
#     'Accept-Encoding': 'gzip, deflate, br',
#     # [{},{}]

def convert_json_to_df(data):
    final_array=[]
    df_column_names = [
    "ticker",
    "name",
    "sector",
    "asset_class",
    "market_value",
    "weight",
    "notional_value",
    "shares",
    "price",
    "location",
    "exchange",
    "currency",
    "fx_rate",
    "market_currency",
    "accrual_date"
]
    for json_data in data:
        try:
            keys=list(json_data.keys())
            if len(keys)==2:
                ticker=json_data[keys[0]] or None
                name=json_data[keys[1]][0] or None
                sector=json_data[keys[1]][1] or None
                asset_class=json_data[keys[1]][2] or None
                market_value=json_data[keys[1]][3] or None
                weight=json_data[keys[1]][4] or None
                notional_value=json_data[keys[1]][5] or None
                shares=json_data[keys[1]][6] or None
                price=json_data[keys[1]][7] or None
                location=json_data[keys[1]][8] or None
                exchange=json_data[keys[1]][9] or None
                currency=json_data[keys[1]][10] or None
                fx_rate=json_data[keys[1]][11] or None
                market_currency=json_data[keys[1]][12] or None
                accrual_date=json_data[keys[1]][13] or None

                final_array.append([ticker,name,sector,asset_class,market_value,weight,notional_value,shares,price,location,exchange,currency,fx_rate,market_currency ,accrual_date])
                
        except Exception as e:
            print("error",json_data)
            return pd.DataFrame({})
    return pd.DataFrame(final_array,columns=df_column_names)




In [56]:



def download_and_save_csv():
    #first we will check if the date is valid to fetch data
     
   
    date=str(datetime.now()).split(" ")[0].split("-")[::-1]
    date="".join(date)

    # URL of the CSV file to download
    # csv_url = 'https://www.ishares.com/us/products/251614/ishares-msci-usa-momentum-factor-etf/1467271812596.ajax?fileType=csv&fileName=MTUM_holdings&dataType=fund&asOfDate={date}'  # Replace with the actual URL
    csv_url=f'https://www.ishares.com/us/products/251614/ishares-msci-usa-momentum-factor-etf/1467271812596.ajax?fileType=csv&fileName=MTUM_holdings&dataType=fund&asOfDate=20240610'

    response = requests.get(csv_url)
      # Check if the request was successful

    # Read CSV content

    csv_file = StringIO(response.text)
    reader = csv.DictReader(csv_file)
    
    

    selected_rows =[]
    start_row=10
    end_row=20
    
    for i, row in enumerate(reader, start=1):
        if start_row <= i:
            selected_rows.append(row)
            
        
    if selected_rows:
        data=convert_json_to_df(selected_rows)
        return data
        
    else:
        return "not updated yet!"
    


In [57]:
data=download_and_save_csv()
data

,ticker,name,sector,asset_class,market_value,weight,notional_value,shares,price,location,exchange,currency,fx_rate,market_currency,accrual_date
0,NVDA,NVIDIA CORP,Information Technology,Equity,"713,222,213.56",6.99,"713,222,213.56","5,856,164.00",121.79,United States,NASDAQ,USD,1.00,USD,-
1,LLY,ELI LILLY,Health Care,Equity,"578,829,455.00",5.67,"578,829,455.00","669,167.00",865.00,United States,New York Stock Exchange Inc.,USD,1.00,USD,-
2,AVGO,BROADCOM INC,Information Technology,Equity,"575,055,790.58",5.63,"575,055,790.58","399,214.00","1,440.47",United States,NASDAQ,USD,1.00,USD,-
3,META,META PLATFORMS INC CLASS A,Communication,Equity,"555,664,005.40",5.44,"555,664,005.40","1,105,579.00",502.60,United States,NASDAQ,USD,1.00,USD,-
4,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,"460,216,983.84",4.51,"460,216,983.84","2,460,264.00",187.06,United States,NASDAQ,USD,1.00,USD,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,USD,USD CASH,Cash and/or Derivatives,Cash,"4,777,698.63",0.05,"4,777,698.63","4,777,699.00",100.00,United States,-,USD,1.00,USD,-
124,IOT,SAMSARA INC CLASS A,Information Technology,Equity,"4,530,027.70",0.04,"4,530,027.70","146,366.00",30.95,United States,New York Stock Exchange Inc.,USD,1.00,USD,-
125,FNF,FIDELITY NATIONAL FINANCIAL INC,Financials,Equity,"2,736,766.08",0.03,"2,736,766.08","55,898.00",48.96,United States,New York Stock Exchange Inc.,USD,1.00,USD,-
126,SGAFT,CASH COLLATERAL USD SGAFT,Cash and/or Derivatives,Cash Collateral and Margins,"918,000.00",0.01,"918,000.00","918,000.00",100.00,United States,-,USD,1.00,USD,"Sep 01, 2016"


In [58]:
data['date']=20230403
data

,ticker,name,sector,asset_class,market_value,weight,notional_value,shares,price,location,exchange,currency,fx_rate,market_currency,accrual_date,date
0,NVDA,NVIDIA CORP,Information Technology,Equity,"713,222,213.56",6.99,"713,222,213.56","5,856,164.00",121.79,United States,NASDAQ,USD,1.00,USD,-,20230403
1,LLY,ELI LILLY,Health Care,Equity,"578,829,455.00",5.67,"578,829,455.00","669,167.00",865.00,United States,New York Stock Exchange Inc.,USD,1.00,USD,-,20230403
2,AVGO,BROADCOM INC,Information Technology,Equity,"575,055,790.58",5.63,"575,055,790.58","399,214.00","1,440.47",United States,NASDAQ,USD,1.00,USD,-,20230403
3,META,META PLATFORMS INC CLASS A,Communication,Equity,"555,664,005.40",5.44,"555,664,005.40","1,105,579.00",502.60,United States,NASDAQ,USD,1.00,USD,-,20230403
4,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,"460,216,983.84",4.51,"460,216,983.84","2,460,264.00",187.06,United States,NASDAQ,USD,1.00,USD,-,20230403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,USD,USD CASH,Cash and/or Derivatives,Cash,"4,777,698.63",0.05,"4,777,698.63","4,777,699.00",100.00,United States,-,USD,1.00,USD,-,20230403
124,IOT,SAMSARA INC CLASS A,Information Technology,Equity,"4,530,027.70",0.04,"4,530,027.70","146,366.00",30.95,United States,New York Stock Exchange Inc.,USD,1.00,USD,-,20230403
125,FNF,FIDELITY NATIONAL FINANCIAL INC,Financials,Equity,"2,736,766.08",0.03,"2,736,766.08","55,898.00",48.96,United States,New York Stock Exchange Inc.,USD,1.00,USD,-,20230403
126,SGAFT,CASH COLLATERAL USD SGAFT,Cash and/or Derivatives,Cash Collateral and Margins,"918,000.00",0.01,"918,000.00","918,000.00",100.00,United States,-,USD,1.00,USD,"Sep 01, 2016",20230403


In [59]:


def fetch_csv_url_from_web(webpage_url):
    try:
        # Make a GET request to the webpage URL
        
        response = requests.get(webpage_url)

        # Check for successful response
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find all anchor tags with .csv links
            csv_links = soup.find_all('a',class_='icon-xls-export')

            # <a class="icon-xls-export" href="/us/products/239726/ishares-core-sp-500-etf/1467271812596.ajax?fileType=csv&amp;fileName=IVV_holdings&amp;dataType=fund" data-link-event="holdings:holdings">Detailed Holdings and Analytics</a>

            # Extract the href attribute from the first .csv link found
            if csv_links:
                csv_link = csv_links[0]['href']
                return csv_link
            else:
                print("No .csv link found on the webpage.")
                return None
        else:
            print(f"Error fetching webpage: {response.status_code}")
            return None
    except Exception as e:
        print(e,'exception occured')
        return None


In [60]:
fetch_csv_url_from_web('https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf')

# https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf/1467271812596.ajax?fileType=csv&fileName=IVV_holdings&dataType=fund

'/us/products/239726/ishares-core-sp-500-etf/1467271812596.ajax?fileType=csv&fileName=IVV_holdings&dataType=fund'

In [62]:
# (data[0])

In [63]:
# [{},{}]
def convert_json_to_df(data):
    final_array=[]
    df_column_names = [
    "ticker",
    "name",
    "sector",
    "asset_class",
    "market_value",
    "weight",
    "notional_value",
    "shares",
    "price",
    "location",
    "exchange",
    "currency",
    "fx_rate",
    "market_currency",
    "accrual_date"
]
    for json_data in data:
        try:
            keys=list(json_data.keys())
            if len(keys)==2:
                ticker=json_data[keys[0]] or None
                name=json_data[keys[1]][0] or None
                sector=json_data[keys[1]][1] or None
                asset_class=json_data[keys[1]][2] or None
                market_value=json_data[keys[1]][3] or None
                weight=json_data[keys[1]][4] or None
                notional_value=json_data[keys[1]][5] or None
                shares=json_data[keys[1]][6] or None
                price=json_data[keys[1]][7] or None
                location=json_data[keys[1]][8] or None
                exchange=json_data[keys[1]][9] or None
                currency=json_data[keys[1]][10] or None
                fx_rate=json_data[keys[1]][11] or None
                market_currency=json_data[keys[1]][12] or None
                accrual_date=json_data[keys[1]][13] or None

                final_array.append([ticker,name,sector,asset_class,market_value,weight,notional_value,shares,price,location,exchange,currency,fx_rate,market_currency ,accrual_date])
                
        except Exception as e:
            print("error",json_data)
            return pd.DataFrame({})
    return pd.DataFrame(final_array,columns=df_column_names)




In [65]:
# convert_json_to_df(data[0])

In [68]:
# type(data[0])

Download csv

In [69]:
def is_valid_etfs(etfs):
     
    #isharedf
    valid_etfs=[]
    for etf in etfs:
        # print(etf)
        
        if etf in  list(ishare_df['ticker']):
            print('valid etf found ',etf)
            valid_etfs.append(etf)
    return valid_etfs
    


In [86]:
def is_valid_date(startdate,enddate):
    # following validation is

    # 0 DD/MM/YYYY FORMAT
    # 1 start <=end date
    # 2 valid dates 
    err=""
    try:
        # Parse dates with flexible formatting for single-digit days/months
        start_date = datetime.strptime(startdate, "%d/%m/%Y")
        end_date = datetime.strptime(enddate, "%d/%m/%Y")
    except ValueError:
         err="Format Error"
         return False,err  # Handle invalid date format

    # Check if start date is less than or equal to end date
    if((start_date <= end_date)==False):
       err='"Start Date must be less than or  equal to end date"'
    return start_date <= end_date,err

    



In [87]:
# is_valid_date('24/02/2020','13/02/2001')

(False, '"Start Date must be less than or  equal to end date"')

Fetch By fund name ,date 

In [88]:
def fetch_data_from_fund(start_date,end_date,fund_name='iShare',etfs=None,all=False):
    # Valid Date checking
    

    valid_date,err= is_valid_date(start_date,end_date)
    if(err):
        print(err)
        return
    
    
    
        
    # Check weather provided ETF exists in our db
    valid_etfs=is_valid_etfs(etfs)
    if not valid_etfs:
        return " error : ETFs not found in our database"
    print( "valid Etfs are "+str(valid_etfs))
    
    # For Perticuler Ticker [ETF] we need to check for its data from link available or not

    for etf in valid_etfs:
        curr_etf_link=ishare_df[ishare_df['ticker']==etf]['link'].values[0]

        try:
            curr_csv_url=fetch_csv_url_from_web(curr_etf_link)+'20240610'
            print(curr_csv_url)
        except Exception as e:
            return "Data is not available for this period..."
            


            

        # print(str(curr_etf_link))
        
    return 
    #extracting csv link from webpage
    # csv_url=fetch_csv_url_from_web("https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf")
    data=download_and_save_csv(csv_url)
    print(data)
    
    
    
    #create function which takes csv link as input and create a dataframe from it
    
    

In [90]:
etfs=["AGG"]
fetch_data_from_fund('23/03/2002','03/02/2022',True,etfs)

valid etf found  AGG
valid Etfs are ['AGG']
/us/products/239458/ishares-core-total-us-bond-market-etf/1467271812596.ajax?fileType=csv&fileName=AGG_holdings&dataType=fund20240610
